In [1]:
# Imports

import pandas as pd
import sklearn as sk
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, normalize
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA

In [2]:
# Read training data
data = pd.read_csv("train.csv")

# Import test data
df = pd.read_csv("test.csv")

# Print length of training data
print(len(data))

# Print first 5 entries of df
data.head()

891


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
data.Cabin.isna().sum()

687

In [4]:
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
data.Survived.sum()/len(data)

0.3838383838383838

In [6]:
print("Missing values per Column:\n")

for col in data.columns:
    print(f'{col}: {data[col].isnull().sum()}')

Missing values per Column:

PassengerId: 0
Survived: 0
Pclass: 0
Name: 0
Sex: 0
Age: 177
SibSp: 0
Parch: 0
Ticket: 0
Fare: 0
Cabin: 687
Embarked: 2


In [7]:
data["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [8]:
## Feature engineering
# PassengerID is redundant
# Name is hard to be a predictor
# Ticket has 681/891 so probably not a predictor
# Cabin a lot of missing values

# Save PassengerID
pID = df["PassengerId"]
print(len(pID))
delete = ["PassengerId", "Name", "Ticket", "Cabin", "Embarked", "Pclass", "Sex", "Age"]

for col in delete:
    del data[col]
    del df[col]

418


In [9]:
# Replace male/female by 0 or 1 in order to be readable by ML model
# Age has a lot of missing values, replace age by avg
# Embarked has 2 missing values, replace by S as it occurs most often

# data['Sex'].replace('female', 0, inplace=True)
# data['Sex'].replace('male', 1, inplace=True)
# data['Age'].fillna(100, inplace=True)
# df['Sex'].replace('female', 0, inplace=True)
# df['Sex'].replace('male', 1, inplace=True)
# df['Age'].fillna(100, inplace=True)
# data['Embarked'].fillna('S', inplace=True)
# df['Embarked'].fillna('S', inplace=True)
# data.head()

In [10]:
data.head()

,Survived,SibSp,Parch,Fare
0,0,1,0,7.2500
1,1,1,0,71.2833
2,1,0,0,7.9250
3,1,1,0,53.1000
4,0,0,0,8.0500


In [11]:
#One-hot encode/make dummies for variable Embarked

# one_hot = pd.get_dummies(data.Embarked, prefix='Embarked')
# data = data.drop('Embarked',axis = 1)
# data = data.join(one_hot)
# one_hot_ = pd.get_dummies(data.Pclass, prefix='Pclass')
# data = data.drop('Pclass',axis = 1)
# data = data.join(one_hot_)
# data.round(1)

In [12]:
#One-hot encode/make dummies for variable Embarked

# one_hotdf = pd.get_dummies(df.Embarked, prefix='Embarked')
# df = df.drop('Embarked',axis = 1)
# df = df.join(one_hotdf)
# one_hot_df = pd.get_dummies(df.Pclass, prefix='Pclass')
# df = df.drop('Pclass',axis = 1)
# df = df.join(one_hot_df)
# df.round(1)

In [13]:
# Convert dataframe to numpy arrays for ML
# Target variable

y = data.Survived.to_numpy()
data = data.drop('Survived', axis = 1)
data.head()

,SibSp,Parch,Fare
0,1,0,7.2500
1,1,0,71.2833
2,0,0,7.9250
3,1,0,53.1000
4,0,0,8.0500


In [14]:
X = data.to_numpy()

In [15]:
# raise NotImplementedError("Stop Running")

In [16]:
# Apply Machine learning (K-fold Crossvalidation)

names = ["Random Forest", "NaiveBayes", "Logistic Regression"]
classifiers = [RandomForestClassifier(n_estimators=10), GaussianNB(), LogisticRegression(random_state=0)]

for i, clf in enumerate(classifiers):
    scores = cross_val_score(clf, X, y, cv=10)
    print(f"{names[i]}: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Random Forest: 0.69 accuracy with a standard deviation of 0.05
NaiveBayes: 0.68 accuracy with a standard deviation of 0.04
Logistic Regression: 0.68 accuracy with a standard deviation of 0.05


In [17]:
df.head()

,SibSp,Parch,Fare
0,0,0,7.8292
1,1,0,7.0000
2,0,0,9.6875
3,0,0,8.6625
4,1,1,12.2875


In [18]:
# Import test data
df.fillna(0, inplace=True)
X2 = df.to_numpy()
X2

array([[ 0.    ,  0.    ,  7.8292],
       [ 1.    ,  0.    ,  7.    ],
       [ 0.    ,  0.    ,  9.6875],
       ...,
       [ 0.    ,  0.    ,  7.25  ],
       [ 0.    ,  0.    ,  8.05  ],
       [ 1.    ,  1.    , 22.3583]])

In [19]:
# predict

rf = RandomForestClassifier(n_estimators=10)
rf.fit(X, y)
pred = rf.predict(X2)

In [20]:
result = pd.DataFrame(columns = ['PassengerId', 'Survived'])
result['PassengerId'] = pID
result['Survived'] = pred
result.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,0
4,896,1


In [21]:
result.to_csv("first_submission.csv", index=False)